# Training Amazon SageMaker models for molecular property prediction by using DGL with PyTorch backend

*이 노트북은 [Training Amazon SageMaker models for molecular property prediction by using DGL with PyTorch backend (영문 원본)](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/dgl_gcn_tox21/pytorch-gcn-tox21.ipynb) 의 한국어 번역입니다.*

**Amazon SageMaker Python SDK**를 사용하면 DGL(Deep Graph Library) 모델을 쉽게 학습할 수 있습니다. 이 노트북에서는 [DGL](https://github.com/dmlc/dgl) 및 Tox21 데이터셋을 사용하여 분자 독성 예측(molecular toxicity prediction)을 위한 간단한 그래프 신경망을 학습합니다.

데이터셋에는 핵 수용체 및 스트레스-반응 경로를 포함하여 12개의 서로 다른 대상(생물 세포 내의 수용체 등)에서 8,014개의 화합물에 대한 정성적인 독성 측정 값이 포함되어 있습니다. 각 대상마다(즉, 12개 레이블에 대한) 이진 분류 문제(화합물에 독성이 있는지 여부)가 발생하며, 그래프 분류(graph classification)로 이 문제를 모델링할 수 있습니다.

DGL 및 GCN(Graph Convolutional Networks)에 대한 자세한 정보는 https://docs.dgl.ai 를 참조하세요.

## Setup

본 예제에서 향후 필요한 기본 변수들을 정의합니다.

In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session

# Setup session
sess = sagemaker.Session()

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sess.default_bucket()

# Location to put your custom code.
custom_code_upload_location = 'customcode'

# IAM execution role that gives Amazon SageMaker access to resources in your AWS account.
# You can use the Amazon SageMaker Python SDK to get the role from the notebook environment. 
role = get_execution_role()

## Training Script

`main.py`는 Amazon SageMaker를 사용하여 분자 특성 예측 모델을 학습하는 데 필요한 모든 코드를 제공합니다.

In [4]:
!pygmentize main.py

import argparse
import dgl
import json
import numpy as np
import os
import random
import torch

from datetime import datetime
from dgl import model_zoo
from dgl.data.chem import Tox21
from dgl.data.utils import split_dataset
from sklearn.metrics import roc_auc_score
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

def setup(args, seed=0):
    args['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Set random seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    return args

def collate_molgraphs(data):
    """Batching a list of datapoints for dataloader."""
    smiles, graphs, labels, masks = map(list, zip(*data))

    bg = dgl.batch(graphs)
    bg.set_n_initializer(dgl.init.zero_initializer)
    bg.set_e_initializer(dgl.init.zero_initializer)
    labels = torch.stack(labels, dim=0)
    masks = torch.stack(masks,

## Bring Your Own Image for Amazon SageMaker

이 예에서는 tox21 데이터셋을 다루기 위해 rdkit 라이브러리가 필요합니다. DGL CPU 및 GPU Docker에는 dgllib 레지스트리 (CPU의 경우 dgllib/dgl-sagemaker-cpu:dgl_0.4_pytorch_1.2.0_rdkit, GPU의 경우 dgllib/dgl-sagemaker-gpu:dgl_0.4_pytorch_1.2.0_rdkit) 아래 Dockerhub에 rdkit 라이브러리가 사전 설치되어 있습니다. GPU의 경우 2.0_rdkit). 요구 사항에 따라 이미지를 직접 가져와서 AWS ECR에 푸시 할 수 있습니다. 아래 스크립트를 참조하시면 도움이 됩니다. Amazon Elastic Container Registry(Amazon ECR)에서 DGL Docker 이미지를 이미 준비한 경우이 이 단계를 건너뛸 수 있습니다.

In [5]:
%%sh
# For CPU default_docker_name="dgllib/dgl-sagemaker-cpu:dgl_0.4_pytorch_1.2.0_rdkit"
default_docker_name="dgllib/dgl-sagemaker-gpu:dgl_0.4_pytorch_1.2.0_rdkit"
docker pull $default_docker_name

docker_name=sagemaker-dgl-pytorch-gcn-tox21

# For CPU docker build -t $docker_name -f gcn_tox21_cpu.Dockerfile .
docker build -t $docker_name -f gcn_tox21_gpu.Dockerfile .

account=$(aws sts get-caller-identity --query Account --output text)
echo $account
region=$(aws configure get region)

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${docker_name}:latest"
# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${docker_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${docker_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

docker tag ${docker_name} ${fullname}
docker push ${fullname}

dgl_0.4_pytorch_1.2.0_rdkit: Pulling from dgllib/dgl-sagemaker-gpu
35c102085707: Pulling fs layer
251f5509d51d: Pulling fs layer
8e829fe70a46: Pulling fs layer
6001e1789921: Pulling fs layer
9f0a21d58e5d: Pulling fs layer
8810fcda1e6e: Pulling fs layer
d701a76e3193: Pulling fs layer
34be232fb7a6: Pulling fs layer
7e62b1ed3410: Pulling fs layer
47526c6630b9: Pulling fs layer
aed822d054ca: Pulling fs layer
4755ed5bed2d: Pulling fs layer
e08ece828d6d: Pulling fs layer
21f7e5dd23c0: Pulling fs layer
8385064a1256: Pulling fs layer
4a916ee6dd88: Pulling fs layer
5224460d79b7: Pulling fs layer
969f958f63e3: Pulling fs layer
031b90ba4752: Pulling fs layer
d9075a74f235: Pulling fs layer
60019ab50995: Pulling fs layer
5225800ad4f4: Pulling fs layer
5b628ec4dfa6: Pulling fs layer
84da67d22223: Pulling fs layer
1d090cebf694: Pulling fs layer
50d54af9151f: Pulling fs layer
b2a664f92319: Pulling fs layer
6001e1789921: Waiting
9f0a21d58e5d: Waiting
8810fcda1e6e: Waiting
d701a76e3193: Waiting
34be232f

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## The Amazon SageMaker Estimator class

Amazon SageMaker Estimator를 사용하면 CPU 또는 GPU 기반 인스턴스를 사용하여 Amazon SageMaker에서 단일 머신을 실행할 수 있습니다.

Estimator 객체를 생성할 때 학습 스크립트의 파일 이름과 IAM 실행 역할의 이름을 전달하고 다른 하이퍼라마테들도 전달하세요. `train_instance_count` 및 `train_instance_type`은 학습 작업에 사용될 SageMaker 인스턴스의 수와 유형을 결정합니다. 하이퍼파라메터는 값들의 딕셔너리를 통해 학습 스크립트에 전달 될 수 있습니다. 처리 방법에 대해서는 `main.py`를 참조하세요.

Amazon SageMaker Docker의 시작점(예: dgllib/dgl-sagemaker-gpu:dgl_0.4_pytorch_1.2.0_rdkit)은 /usr/bin/에 있는 학습 스크립트입니다. 위에 제공된 dgl docker 이미지 내부의 학습 스크립트는 하이퍼파라메터(키 'entrypoint'포함)에서 실제 진입점(entrypoint)을 가져오고 'training-code' 데이터 채널 (/opt/ml/input/data/training-code/)의 실제 진입점을 실행합니다.

이 예시에서는 ml.p3.2xlarge 단일 인스턴스 선택합니다. CPU 이미지에 ml.c4.2xlarge와 같은 CPU 인스턴스를 사용할 수도 있습니다.

In [6]:
import boto3

# Set target dgl-docker name
docker_name='sagemaker-dgl-pytorch-gcn-tox21'

CODE_PATH = 'main.py'
code_location = sess.upload_data(CODE_PATH, bucket=bucket, key_prefix=custom_code_upload_location)

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, docker_name)
print(image)

estimator = sagemaker.estimator.Estimator(image,
                        role, 
                        train_instance_count=1, 
                        train_instance_type= 'ml.p3.2xlarge', #'ml.c4.2xlarge'
                        hyperparameters={'entrypoint': CODE_PATH},
                        sagemaker_session=sess)

143656149352.dkr.ecr.us-east-1.amazonaws.com/sagemaker-dgl-pytorch-gcn-tox21:latest


## Running the Training Job

Estimator 객체를 생성한 후 Amazon SageMaker를 사용하여 학습을 수행합니다.

In [7]:
estimator.fit({'training-code': code_location})

2019-12-10 01:19:53 Starting - Starting the training job...
2019-12-10 01:19:54 Starting - Launching requested ML instances......
2019-12-10 01:20:58 Starting - Preparing the instances for training......
2019-12-10 01:22:10 Downloading - Downloading input data
2019-12-10 01:22:10 Training - Downloading the training image............../opt/ml/input/data/training-code /

2019-12-10 01:24:35 Training - Training image download completed. Training in progress.Downloading /root/.dgl/tox21.csv.gz from https://s3.us-east-2.amazonaws.com/dgl.ai/dataset/tox21.csv.gz...
epoch 1/100, batch 1/49, loss 0.9754
epoch 1/100, batch 2/49, loss 1.0454
epoch 1/100, batch 3/49, loss 1.0320
epoch 1/100, batch 4/49, loss 0.9964
epoch 1/100, batch 5/49, loss 1.0340
epoch 1/100, batch 6/49, loss 0.9648
epoch 1/100, batch 7/49, loss 1.0190
epoch 1/100, batch 8/49, loss 1.0844
epoch 1/100, batch 9/49, loss 0.9548
epoch 1/100, batch 10/49, loss 0.9281
epoch 1/100, batch 11/49, loss 1.0733
epoch 1/100, batch 12/49,

epoch 9/100, batch 17/49, loss 0.7853
epoch 9/100, batch 18/49, loss 0.6108
epoch 9/100, batch 19/49, loss 0.6860
epoch 9/100, batch 20/49, loss 0.5957
epoch 9/100, batch 21/49, loss 0.6221
epoch 9/100, batch 22/49, loss 0.6385
epoch 9/100, batch 23/49, loss 0.5760
epoch 9/100, batch 24/49, loss 0.6419
epoch 9/100, batch 25/49, loss 0.5932
epoch 9/100, batch 26/49, loss 0.6511
epoch 9/100, batch 27/49, loss 0.5659
epoch 9/100, batch 28/49, loss 0.6925
epoch 9/100, batch 29/49, loss 0.5447
epoch 9/100, batch 30/49, loss 0.6610
epoch 9/100, batch 31/49, loss 0.6588
epoch 9/100, batch 32/49, loss 0.6487
epoch 9/100, batch 33/49, loss 0.7229
epoch 9/100, batch 34/49, loss 0.6839
epoch 9/100, batch 35/49, loss 0.6898
epoch 9/100, batch 36/49, loss 0.5767
epoch 9/100, batch 37/49, loss 0.6492
epoch 9/100, batch 38/49, loss 0.5253
epoch 9/100, batch 39/49, loss 0.5585
epoch 9/100, batch 40/49, loss 0.5324
epoch 9/100, batch 41/49, loss 0.6332
epoch 9/100, batch 42/49, loss 0.6213
epoch 9/100,

epoch 17/100, batch 24/49, loss 0.5066
epoch 17/100, batch 25/49, loss 0.5107
epoch 17/100, batch 26/49, loss 0.4154
epoch 17/100, batch 27/49, loss 0.4421
epoch 17/100, batch 28/49, loss 0.5186
epoch 17/100, batch 29/49, loss 0.4352
epoch 17/100, batch 30/49, loss 0.5090
epoch 17/100, batch 31/49, loss 0.5325
epoch 17/100, batch 32/49, loss 0.4106
epoch 17/100, batch 33/49, loss 0.4854
epoch 17/100, batch 34/49, loss 0.5110
epoch 17/100, batch 35/49, loss 0.4649
epoch 17/100, batch 36/49, loss 0.5164
epoch 17/100, batch 37/49, loss 0.4332
epoch 17/100, batch 38/49, loss 0.4481
epoch 17/100, batch 39/49, loss 0.6129
epoch 17/100, batch 40/49, loss 0.4374
epoch 17/100, batch 41/49, loss 0.4955
epoch 17/100, batch 42/49, loss 0.5914
epoch 17/100, batch 43/49, loss 0.5100
epoch 17/100, batch 44/49, loss 0.4517
epoch 17/100, batch 45/49, loss 0.4874
epoch 17/100, batch 46/49, loss 0.5389
epoch 17/100, batch 47/49, loss 0.5277
epoch 17/100, batch 48/49, loss 0.5032
epoch 17/100, batch 49/49


2019-12-10 01:25:37 Uploading - Uploading generated training model
2019-12-10 01:25:37 Completed - Training job completed
Training seconds: 220
Billable seconds: 220


## Output
학습 완료 후, 학습 작업을 검색하고 'S3 모델 아티팩트(S3 model artifact)'주소를 찾아 Amazon Sagemaker 콘솔에서 모델 학습 결과를 얻을 수 있습니다.